In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import json
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

In [3]:
import tensorflow as tf

tf.__version__

'2.14.0'

In [4]:
validation_data = []

val_folder_path = '/content/drive/MyDrive/NLP/Project/data/validation_dataset'

In [5]:
for item in os.listdir(val_folder_path):
    with open(os.path.join(val_folder_path, item), 'r') as f:
        data = json.load(f)
        validation_data.extend(data)

In [6]:
print(len(validation_data))

240


In [7]:
df_val = pd.DataFrame(validation_data)

In [8]:
df_val.sample(1)

,Story,Question,Answer Choices,Answer,Reasoning
226,"In Rome, Italy, an international tribunal star...",Which court held the session after the discove...,"[French Supreme Court, Paris, France, Internat...","French Supreme Court, Paris, France",The story mentions that after a key document w...


In [9]:
choices = df_val['Answer Choices'].apply(lambda x: list(x))
df_val[['A', 'B', 'C', 'D']] = pd.DataFrame(choices.tolist(), index=choices.index)

In [10]:
prompt = []
for item in df_val.iterrows():
    story = item[1]['Story']
    question = item[1]['Question']
    fullprompt = story + ' ' + question
    prompt.append(fullprompt)

df_val['prompt'] = prompt

In [11]:
options = ['A', 'B', 'C', 'D']

mapping = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
reverse_mapping = {0: 'A', 1:'B', 2:'C', 3:'D'}

In [12]:
labels = []
for item in df_val.iterrows():
    answer = item[1]['Answer']
    choices = item[1]['Answer Choices']
    label = choices.index(answer)
    option = mapping[reverse_mapping[label]]
    labels.append(option)

df_val['answer'] = labels

In [13]:
df_val.sample(1)

,Story,Question,Answer Choices,Answer,Reasoning,A,B,C,D,prompt,answer
204,"On January 15, 2019, in Melbourne, Australia, ...",How long did 'The Savoy Affair' last from star...,"[Eight months and 15 days, Six months, Seven m...",Eight months and 15 days,"The trial started on January 15, 2019, and the...",Eight months and 15 days,Six months,Seven months,Eight months,"On January 15, 2019, in Melbourne, Australia, ...",0


In [14]:
new_df_val = df_val.drop(['Story', 'Question', 'Answer Choices', 'Reasoning', 'Answer'], axis=1)

In [15]:
new_df_val.sample(1)

,A,B,C,D,prompt,answer
19,2nd July,3rd July,4th July,5th July,"Every year, the small town of Brody, nestled i...",2


In [16]:
new_df_val.shape

(240, 6)

In [17]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00


In [18]:
from datasets import Dataset

datasets = Dataset.from_pandas(new_df_val)
datasets = datasets.rename_column("answer", "label")
datasets

Dataset({
    features: ['A', 'B', 'C', 'D', 'prompt', 'label'],
    num_rows: 240
})

In [19]:
datasets[0]

{'A': 'Less than a month',
 'B': 'About a month',
 'C': 'More than a month',
 'D': 'Two months',
 'prompt': 'In February, Ramesh, a painter from Delhi, travelled to Uttar Pradesh to attend the Lucknow Literature Festival, which was renowned for showcasing several art installations at well-known sites across the city. The event, taking place from 7th to 14th of February, brought together both art and literary enthusiasts from all corners of the country. After successfully showcasing his work at the festival, Ramesh took a short trip to Agra to visit the Taj Mahal before returning to Delhi mid-March. In May, he decided to attend the much acclaimed Ziro Festival in Arunachal Pradesh. The festival, happening in between 20th and 27th May, was an exceptional combination of music and visual art involving artists from diverse backgrounds, and after participating, he finally returned to Delhi at the end of June. How much time did Ramesh spend in Uttar Pradesh in total between attending the fest

In [20]:
!pip install transformers

In [21]:
from transformers import AutoTokenizer, TFAutoModelForMultipleChoice
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")
model = TFAutoModelForMultipleChoice.from_pretrained("bert-large-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForMultipleChoice.

Some weights or buffers of the TF 2.0 model TFBertForMultipleChoice were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
predicted_label = []
ground_truth = []
for i in tqdm(range(len(datasets))):
    prompt = datasets[i]['prompt']
    candidate1 = datasets[i]['A']
    candidate2 = datasets[i]['B']
    candidate3 = datasets[i]['C']
    candidate4 = datasets[i]['D']
    ground_truth.append(datasets[i]['label'])
    inputs = tokenizer([[prompt, candidate1], [prompt, candidate2], [prompt, candidate3], [prompt, candidate4]], return_tensors="tf", padding=True)
    inputs = {k: tf.expand_dims(v, 0) for k, v in inputs.items()}
    outputs = model(inputs)
    logits = outputs.logits
    predicted_class = int(tf.math.argmax(logits, axis=-1)[0])
    predicted_label.append(predicted_class)

100%|██████████| 240/240 [01:22<00:00,  2.90it/s]


In [23]:
from sklearn.metrics import classification_report

report = classification_report(ground_truth, predicted_label)
print(report)

              precision    recall  f1-score   support

           0       0.17      0.23      0.20        43
           1       0.39      0.24      0.30        86
           2       0.26      0.21      0.23        71
           3       0.13      0.23      0.16        40

    accuracy                           0.23       240
   macro avg       0.24      0.23      0.22       240
weighted avg       0.27      0.23      0.24       240

